In [2]:
from PIL import Image
import torch
import torchvision.transforms as transforms
from transformers import ViTFeatureExtractor
from torchvision.transforms import (CenterCrop, 
                                    Compose, 
                                    Normalize, 
                                    RandomHorizontalFlip,
                                    RandomResizedCrop, 
                                    Resize, 
                                    ToTensor)
from torch.utils.data import DataLoader
from transformers import ViTForImageClassification
from transformers import TrainingArguments, Trainer
from datasets import load_metric
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

ModuleNotFoundError: No module named 'torch'

In [ ]:
class Dataloader:
    def __init__(self, df):
        self.img_path_list = df["img_path_list"].values
        self.labels = df["label"].values
        
    def __call__(self):
        for img_path, label in zip(self.img_path_list, self.labels):
            img_list = []
            for single_path in img_path:
                if single_path is not None:
                    img = Image.load(single_path)
                    img2 = cv2.imread(single_path)
                    print(img.shape, img2.shape)
                    

In [ ]:
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
_train_transforms = Compose(
        [
            RandomResizedCrop(feature_extractor.size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

_val_transforms = Compose(
        [
            Resize(feature_extractor.size),
            CenterCrop(feature_extractor.size),
            ToTensor(),
            normalize,
        ]
    )

def train_transforms(examples):
    examples['pixel_values'] = [_train_transforms(image.convert("RGB")) for image in examples['img']]
    return examples

def val_transforms(examples):
    examples['pixel_values'] = [_val_transforms(image.convert("RGB")) for image in examples['img']]
    return examples